In [618]:
import pandas as pd
import requests
import yfinance as yf
import investpy
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [619]:
class Accion:
    def __init__(self, mercado, datosAccion):
        self.nombre = datosAccion[mercado.campo_nombre]
        self.max = datosAccion[mercado.campo_valorMaximo]
        self.min = datosAccion[mercado.campo_valorMinimo]
        self.ult = datosAccion[mercado.campo_UltimoPrecio]
        self.mov = datosAccion[mercado.campo_cambio]
        
class Origen:
    def __init__(self):
       self.__abrirColeccion()

    def __abrirColeccion(self):
        client = MongoClient('localhost')
        bd = client['cotizaciones']
        self.coleccion = bd[self.__class__.__name__]
        
    def _guardarEnBD(self, dataFrame):
        self.coleccion.drop()
        self.coleccion.insert_many(dataFrame.to_dict('records'))
        
    def visualizarBD(self):
        for documento in self.coleccion.find({}):
            print(documento)
            
    def visualizarDF(self):
        df = self.__documentADF(self.coleccion.find({}))
        print (df)

    def __documentADF(self, listColeccion):
        df = pd.DataFrame(list(listColeccion))
        return df.drop(['_id'], axis=1)
        
    def __cotizacionMinima(self):
        pass
    def __cotizacionMaxima(self):
        pass

    def filtrarAcciones(self, nombreAccion):
        accion = self.coleccion.find({self.campo_nombre: {"$in": nombreAccion}})
        return print(self.__documentADF(accion))

    def accion(self, symbol):
        return Accion(self, self.coleccion.find_one({self.campo_nombre: symbol}))

In [620]:
       
class bolsaMadrid(Origen):
    def __init__(self):
        super().__init__()
        self.campo_nombre = 'Nombre'
        self.campo_valorMaximo = 'Máx'
        self.campo_valorMinimo = 'Mín'
        self.campo_UltimoPrecio = 'Últ'
        self.campo_cambio = '% Dif'
    
    def actualizarBD(self):
        #Hacemos la consulta a la pagina y guardamos el contenido en una variable soup
        url_page = 'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'
        page = requests.get(url_page).text 
        soup = BeautifulSoup(page, "lxml")

        #Buscamos la tabla de donde obtener los datos
        tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})

        #creamos una lista vacia que llenaremos con los datos de las tablas
        columnas = []

        #buscamos los encabezados de la tabla y los agregamos a la lista columnos (deberia ubicarse en la posicion 0)
        headers = []
        for th in tabla.find_all('th'):
            headers.append(th.text.replace('.',''))
        columnas.append(headers)

        for tr in tabla.find_all('tr'):
            fila = []
            for td in tr.find_all('td'):
                fila.append(td.text)
            if len(fila) > 0:
                columnas.append(fila)
                
        #creamos el dataframe usando la lista columnas,pasamos el contenido (de la posicion 1 en adelante), y lo encabezados en (posicion 0)
        df = pd.DataFrame(columnas[1:], columns=columnas[0])

        #Convertirmos a float
        df = self.__convertirAFloat(df)

        #Guardamos los datos en la base de datos
        self._guardarEnBD(df)
        
    def __convertirAFloat(self, df):
        df["% Dif"] = df["% Dif"].str.replace(",", ".").astype(float)
        df["Máx"] = df["Máx"].str.replace(",", ".").astype(float)
        df["Mín"] = df["Mín"].str.replace(",", ".").astype(float)
        df["Últ"] = df["Últ"].str.replace(",", ".").astype(float)
        return df

nombreDeAcciones =[
    'BBVA', "BBVA.MC",
    'ELE', "ELE.MC", 'ENDESA',
    'GRLS', "GRF.MC", 'GRIFOLS CL.A',
    'IDR', "IDR.MC", 'INDRA A'
    'REP', "REP.MC", 'REPSOL'
    ]

bolsamadrid = bolsaMadrid()
bolsamadrid.actualizarBD()
#bolsamadrid.visualizarDF()
bolsamadrid.filtrarAcciones(nombreDeAcciones)
accion = bolsamadrid.accion('BBVA')
print(accion.mov)

         Nombre     Últ  % Dif     Máx     Mín    Volumen Efectivo (miles €)  \
0          BBVA   6.170  -1.74   6.254   6.141  7.296.520          45.124,14   
1        ENDESA  20.060  -0.10  20.140  19.960    448.554           8.994,62   
2  GRIFOLS CL.A  19.760  -0.08  19.815  19.595    473.007           9.313,75   
3        REPSOL  10.882  -1.23  11.000  10.764  3.045.917          33.059,05   

        Fecha      Hora  
0  02/11/2021  16:13:11  
1  02/11/2021  16:12:31  
2  02/11/2021  16:11:58  
3  02/11/2021  16:13:07  
-1.74


In [621]:
class yfinance(Origen):

    def __init__(self):
        super().__init__()
        self.campo_nombre = 'Ticker'
        self.campo_valorMaximo = 'High'
        self.campo_valorMinimo = 'Low'
        self.campo_UltimoPrecio = 'Close'
        self.campo_cambio = 'Dif'

    def actualizarBD(self, acciones):
        df_list = list()
        for accion in acciones:
            if not " " in accion and ".MC" in accion:
                data = yf.download(accion, group_by='ticker', period='1d')
                if not data.empty:
                    data['Ticker'] = accion
                    data['Dif'] = self.__calcularDiferencia(data['Open'], data['Close'])
                    df_list.append(data)

        if len(df_list) > 0:
            df = pd.concat(df_list)
            self._guardarEnBD(df)
    
    def __calcularDiferencia(self, A, B):
        return (B*100)/A-100

yahooFinance = yfinance()
yahooFinance.actualizarBD(nombreDeAcciones)
yahooFinance.visualizarBD()
yahooFinance.visualizarDF()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
{'_id': ObjectId('618159315bc14ce365e80f12'), 'Open': 6.202000141143799, 'High': 6.254000186920166, 'Low': 6.140999794006348, 'Close': 6.173999786376953, 'Adj Close': 6.173999786376953, 'Volume': 7315483, 'Ticker': 'BBVA.MC', 'Dif': -0.4514729785491767}
{'_id': ObjectId('618159315bc14ce365e80f13'), 'Open': 20.040000915527344, 'High': 20.139999389648438, 'Low': 19.959999084472656, 'Close': 20.059999465942383, 'Adj Close': 20.059999465942383, 'Volume': 448569, 'Ticker': 'ELE.MC', 'Dif': 0.09979316118466386}
{'_id': ObjectId('618159315bc14ce365e80f14'), 'Open': 19.799999237060547, 'High': 19.815000534057617, 'Low': 19.594999313354492, 'Close': 19.760

In [622]:
class investPy(Origen):

    def __init__(self):
        super().__init__()
        self.campo_nombre = 'symbol'
        self.campo_valorMaximo = 'high'
        self.campo_valorMinimo = 'low'
        self.campo_UltimoPrecio = 'last'
        self.campo_cambio = 'DP'

    def actualizarBD(self):
        df = investpy.stocks.get_stocks_overview(country='spain', as_json = False , n_results = 35)  
        df = self.__porcentajeCambio(df)
        #cambiar la columna a numérica
        df['GP'] = df['GP'].apply(pd.to_numeric)
        self._guardarEnBD(df)
    
    def __porcentajeCambio(self, df):
        return df.join(df['change_percentage'].str.partition('%')[[0, 1]]).rename({0: 'GP', 1: '%'}, axis=1)

invest = investPy()
invest.actualizarBD()
invest.filtrarAcciones(nombreDeAcciones)
""" invest.visualizarBD()
invest.visualizarDF() """


  country     name symbol    last    high     low  change change_percentage  \
0   spain     BBVA   BBVA   6.174   6.254   6.140  -0.105            -1.67%   
1   spain  Grifols   GRLS  19.744  19.809  19.602  +0.014            +0.07%   
2   spain  Indra A    IDR  10.420  10.460  10.295  +0.010            +0.10%   
3   spain   Endesa    ELE  20.119  20.135  19.954  +0.039            +0.19%   

   turnover currency    GP  %  
0   7320000      EUR -1.67  %  
1    474400      EUR  0.07  %  
2    288790      EUR  0.10  %  
3    448570      EUR  0.19  %  


' invest.visualizarBD()\ninvest.visualizarDF() '

In [624]:
class Cotizaciones(bolsaMadrid, yfinance, investPy):

    def __init__(self, acciones):
        self.acciones = acciones
        self.actualizarBD()
       

    def actualizarBD(self):
        bolsaMadrid.obtenerDatos(self)
        yfinance.obtenerDatos(self.acciones)
        ##investpy.obtenerDatos(self)

    def leerDatos():
        pass

In [625]:
acciones = ['ACCIONA', 'BBVA', 'GRIFOLS CL.A', 'PEPITO', 'REPSOL', 'ENDESA']
cotizaciones = Cotizaciones(acciones)
cotizaciones.mostrarDatosEnPantalla()


AttributeError: type object 'bolsaMadrid' has no attribute 'obtenerDatos'